## First Neural Network: Image Classification 

Objectives:
- Train a minimal image classifier on [MNIST](https://paperswithcode.com/dataset/mnist) using PyTorch
- Usese PyTorch and torchvision

In [1]:
# The usual imports

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# load the data

class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        return torch.reshape(img, self.new_size)

transformations = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.ConvertImageDtype(torch.float32),
                                ReshapeTransform((-1,))
                                ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transformations)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transformations)

In [3]:
# check shape of data

trainset.data.shape, testset.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [4]:
# data loader

BATCH_SIZE = 128
train_dataloader = torch.utils.data.DataLoader(trainset, 
                                               batch_size=BATCH_SIZE,
                                               shuffle=True, 
                                               num_workers=0)

test_dataloader = torch.utils.data.DataLoader(testset, 
                                              batch_size=BATCH_SIZE,
                                              shuffle=False, 
                                              num_workers=0)

In [5]:
# model

model = nn.Sequential(nn.Linear(784, 512), nn.ReLU(), nn.Linear(512, 10))

In [6]:
# training preparation

trainer = torch.optim.RMSprop(model.parameters())
loss = nn.CrossEntropyLoss()

In [7]:
def get_accuracy(output, target, batch_size):
    # Obtain accuracy for training round
    corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [8]:
# train

for ITER in range(5):
    train_acc = 0.0
    train_running_loss = 0.0

    model.train()
    for i, (X, y) in enumerate(train_dataloader):
        output = model(X)
        l = loss(output, y)

        # update the parameters
        l.backward()
        trainer.step()
        trainer.zero_grad()

        # gather metrics
        train_acc += get_accuracy(output, y, BATCH_SIZE)
        train_running_loss += l.detach().item()

    print('Epoch: %d | Train loss: %.4f | Train Accuracy: %.4f' \
          %(ITER+1, train_running_loss / (i+1),train_acc/(i+1)))

Epoch: 1 | Train loss: 0.9943 | Train Accuracy: 91.7344
Epoch: 2 | Train loss: 0.1334 | Train Accuracy: 95.9422
Epoch: 3 | Train loss: 0.1030 | Train Accuracy: 96.8767
Epoch: 4 | Train loss: 0.0845 | Train Accuracy: 97.4997
Epoch: 5 | Train loss: 0.0735 | Train Accuracy: 97.8811


### Other things to try

- Evaluate on test set
- Plot loss curve
- Add more layers to the model